In [1]:
import os
from unityagents import UnityEnvironment
import numpy as np
import torch
from ddpg_agent import Agent
from maddpg import MaddpgAgent


In [2]:
root_dir = "C:/Users/johnb/repos/Udacity/deep-reinforcement-learning/p3_collab-compet/Tennis_Windows_x86_64"
env = UnityEnvironment(file_name=root_dir+"/Tennis.exe")
log_path = os.getcwd()+"/log"
model_dir= os.getcwd()+"/model_dir"

os.makedirs(model_dir, exist_ok=True)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [5]:
# for i in range(num_agents):

#     save_dict = {'actor_params' : maddpg_agent.agents[i].actor_local.state_dict(),
#                 'actor_optim_params': maddpg_agent.agents[i].actor_optimizer.state_dict(),
#                 'critic_params' : maddpg_agent.agents[i].critic_local.state_dict(),
#                 'critic_optim_params' : maddpg_agent.agents[i].critic_optimizer.state_dict()}
#     save_dict_list.append(save_dict)

#     torch.save(save_dict_list, 
#             os.path.join(model_dir, 'episode-{}.pt'.format(episode)))
    
agent = Agent(state_size, action_size, 42, 0.1)
actor = agent.actor_local

model_params = torch.load(os.path.join(model_dir, 'episode-{}.pt'.format(900)))

actor.load_state_dict(model_params[0]['actor_params'])

In [6]:
for i in range(5):                                         # play game for 5 episodes
        env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        while True:
        #     actions = np.random.randn(num_agents, action_size) # select an action (for each agent) 
            state_tensor =  torch.tensor(states, dtype=torch.float32)
            actions = actor(state_tensor).detach().numpy()     
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            scores += env_info.rewards                         # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

    

env.close()

Total score (averaged over agents) this episode: 1.2950000194832683
Total score (averaged over agents) this episode: 2.600000038743019
